In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,10769
2,Huelva,Confirmados PDIA 14 días,980
3,Huelva,Tasa PDIA 14 días,"191,877323820395"
4,Huelva,Confirmados PDIA 7 días,483
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,1
629,Municipio de Huelva sin especificar,Total Confirmados,123
630,Municipio de Huelva sin especificar,Curados,101


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  10769.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  3045.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1012 personas en los últimos 7 días 

Un positivo PCR cada 489 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,10769.0,483.0,980.0,510743.0,94.568110,191.877324,82.0
Huelva-Costa,6049.0,309.0,650.0,288115.0,107.248842,225.604359,59.0
Huelva (capital),3045.0,142.0,294.0,143663.0,98.842430,204.645594,30.0
Condado-Campiña,3315.0,153.0,296.0,155057.0,98.673391,190.897541,23.0
Cartaya,385.0,36.0,69.0,19974.0,180.234305,345.449084,11.0
Aljaraque,422.0,31.0,55.0,21260.0,145.813735,258.701787,8.0
Moguer,367.0,30.0,64.0,22088.0,135.820355,289.750091,5.0
Palos de la Frontera,233.0,17.0,32.0,11289.0,150.589069,283.461777,4.0
Paterna del Campo,68.0,8.0,15.0,3509.0,227.985181,427.472214,4.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Palma del Condado (La),519.0,37.0,73.0,10761.0,343.834216,678.375616,1.0
Jabugo,46.0,11.0,12.0,2250.0,488.888889,533.333333,0.0
Lucena del Puerto,95.0,12.0,17.0,3371.0,355.977455,504.301394,2.0
Isla Cristina,410.0,34.0,104.0,21264.0,159.894658,489.089541,2.0
Paterna del Campo,68.0,8.0,15.0,3509.0,227.985181,427.472214,4.0
Cartaya,385.0,36.0,69.0,19974.0,180.234305,345.449084,11.0
Lepe,943.0,40.0,82.0,27431.0,145.820422,298.931865,3.0
Moguer,367.0,30.0,64.0,22088.0,135.820355,289.750091,5.0
Palos de la Frontera,233.0,17.0,32.0,11289.0,150.589069,283.461777,4.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Villalba del Alcor,103.0,1.0,7.0,3338.0,29.958059,209.706411,0.0,0.142857
San Bartolomé de la Torre,45.0,1.0,7.0,3736.0,26.766595,187.366167,0.0,0.142857
Gibraleón,349.0,2.0,11.0,12607.0,15.864202,87.253113,0.0,0.181818
Punta Umbría,232.0,5.0,16.0,15242.0,32.804094,104.973101,0.0,0.312500
Isla Cristina,410.0,34.0,104.0,21264.0,159.894658,489.089541,2.0,0.326923
Zalamea la Real,47.0,1.0,3.0,3068.0,32.594524,97.783572,0.0,0.333333
Minas de Riotinto,67.0,1.0,3.0,3848.0,25.987526,77.962578,0.0,0.333333
Moguer,367.0,30.0,64.0,22088.0,135.820355,289.750091,5.0,0.468750
Huelva-Costa,6049.0,309.0,650.0,288115.0,107.248842,225.604359,59.0,0.475385
